In [3]:
from fk import *

generate data set

In [4]:
import numpy as np

def generate_dataset(num_samples, n, a, d, alpha):
    """
    Generate a dataset of joint angles and corresponding end-effector positions.

    Parameters:
    num_samples (int): Number of samples to generate.
    n (int): Number of links in the manipulator.
    a (list): List of link lengths.
    d (list): List of link offsets.
    alpha (list): List of constant link twists.

    Returns:
    X (numpy.ndarray): End-effector positions (num_samples, 3).
    y (numpy.ndarray): Joint angles (num_samples, n).
    """
    X = []
    y = []

    for _ in range(num_samples):
        # Randomly generate joint angles (theta)
        theta = np.random.uniform(-np.pi, np.pi, size=n)

        # Compute the end-effector position using forward kinematics
        T = fk(n, theta, alpha, a, d)
        end_effector_pos = T[:3, 3]  # Extract x, y, z from the transformation matrix

        # Append to dataset
        X.append(end_effector_pos)
        y.append(theta)

    return np.array(X), np.array(y)

# Define robot parameters
n = 7  # Number of links
a = [0, 218,196,36,0,7.5,11]  # Link lengths
d = [147,0,0,0,425,0,0]  # Link offsets
alpha = [np.pi/2,0,0,np.pi/2,-np.pi/2,-np.pi/2,0]  # Constant link twists

# Generate dataset
num_samples = 100000
X, y = generate_dataset(num_samples, n, a, d, alpha)



split into training, validation, and test sets

In [5]:
from sklearn.model_selection import train_test_split

# First split: Split the dataset into training (80%) and temporary set (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Second split: Split the temporary set into validation (50%) and test (50%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the shapes of the resulting sets
print("Training set shapes:", X_train.shape, y_train.shape)
print("Validation set shapes:", X_val.shape, y_val.shape)
print("Test set shapes:", X_test.shape, y_test.shape)

Training set shapes: (80000, 3) (80000, 7)
Validation set shapes: (10000, 3) (10000, 7)
Test set shapes: (10000, 3) (10000, 7)


generate model

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(input_shape, output_shape):
    """
    Build a neural network model for inverse kinematics.

    Parameters:
    input_shape (int): Number of input features (3 for x, y, z).
    output_shape (int): Number of output features (n for joint angles).

    Returns:
    model (tf.keras.Model): Compiled neural network model.
    """
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(output_shape)  # Output layer predicts joint angles
    ])

    model.compile(optimizer='adam', loss='mse')  # Mean squared error loss
    return model

# Build the model
input_shape = X_train.shape[1]  # 3 (x, y, z)
output_shape = y_train.shape[1]  # n (joint angles)
model = build_model(input_shape, output_shape)

# Print model summary
model.summary()

a:\pycharm\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,287 (67.53 KB)

 Trainable params: 17,287 (67.53 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32
)

Epoch 1/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 656us/step - loss: 17.8935 - val_loss: 3.4137
Epoch 2/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 606us/step - loss: 3.3253 - val_loss: 3.1643
Epoch 3/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 623us/step - loss: 3.1569 - val_loss: 3.1160
Epoch 4/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 629us/step - loss: 3.1245 - val_loss: 3.1106
Epoch 5/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 600us/step - loss: 3.1170 - val_loss: 3.1056
Epoch 6/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 609us/step - loss: 3.1128 - val_loss: 3.1004
Epoch 7/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 614us/step - loss: 3.0824 - val_loss: 3.0390
Epoch 8/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 617us/step - loss: 3.0516 - val_loss: 3.0343
Epoch 9/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 622us/step - loss: 3.0418 - val_loss: 3.0462
Epoch 10/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 636us/step - loss: 3.0318 - val_loss: 3.0289
Epoch 11/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 614us/step - loss: 3.0294 - val_loss: 3.0